In [814]:
# import comet_ml in the top of your file
from comet_ml import Experiment
   

In [815]:
 
# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="toXXRULujXVmmXBW2zMcncxEI",
                        project_name="liw", workspace="shyken",
                        display_summary_level=0)

ImportError: You must import Comet before these modules: sklearn

### Library Imports

In [ ]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
import reverse_geocode
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from info_gain import info_gain
from sklearn.model_selection import train_test_split
from geopy.distance import great_circle
import statistics
import pickle

%matplotlib inline

In [ ]:
import reverse_geocoder as rg 
import pprint 

### Train dataset with known Geolocation

In [ ]:
list_of_lists = []

with open('full_text.txt','r',encoding="latin-1") as f:
    for line in f:
        inner_list = [line.strip() for line in line.split('\t')]
        list_of_lists.append(inner_list)

df = pd.DataFrame(list_of_lists)

In [ ]:
df

In [ ]:
cols = {0:'user_id',1:'date_time',2:'location',3:'lat',4:'long',5:'tweet'}

df.rename(columns=cols,inplace=True)

In [ ]:
df['location'] = df['location'].apply(lambda x:x[5:])

In [ ]:
df

In [ ]:
def char(loc):
    if re.search(r"\S?e:",loc) != None:
        return loc[2:].replace(' ','')
    else:
        return loc

In [ ]:
df.location = df.location.apply(lambda x: char(x))

In [486]:
def decifer(df):
    copy = df.iloc[:5000].copy()
    decoded = []
    i = 0
    for loc in copy.location:

        lat = float(loc.split(',')[0])
        lon = float(loc.split(',')[1])
        dec = rg.search((lat,lon))[0]
        decoded.append(str(dec['admin1'])+' - '+ str(dec['cc']))
        print([str(dec['admin1'])+' - '+ str(dec['cc']),str(i)])
        i=i+1 
        
    return decoded

In [487]:
decoded_loc_ = decifer(df)

Loading formatted geocoded file...


KeyboardInterrupt: 

In [ ]:
def decifer_s5000(df):
    copy = df.iloc[5000:10000].copy()
    decoded = []
    i = 0
    for loc in copy.location:

        lat = float(loc.split(',')[0])
        lon = float(loc.split(',')[1])
        dec = rg.search((lat,lon))[0]
        decoded.append(str(dec['admin1'])+' - '+ str(dec['cc']))
        print([str(dec['admin1'])+' - '+ str(dec['cc']),str(i)])
        i=i+1 
        
    return decoded

In [ ]:
decoded_loc_s5000 = decifer_s5000(df)

In [816]:
sub_df = pd.read_csv('saved_output.csv')

In [817]:
sub_df

,0
0,Washington - US
1,Washington - US
2,Washington - US
3,Washington - US
4,Washington - US
...,...
9995,Illinois - US
9996,Illinois - US
9997,Illinois - US
9998,Illinois - US


In [818]:
df['decoded']=sub_df

In [819]:
df.dropna(inplace=True)

In [820]:
df.drop(index=3928,inplace=True)

KeyError: '[3928] not found in axis'

In [822]:
ind = list(df[df.decoded=='New York - US'].iloc[:100,:].index)

In [823]:
df.drop(index=ind,inplace=True)

In [824]:
df.dropna(inplace=True)

In [825]:
df.decoded.value_counts()

New York - US            1854
Texas - US                862
Georgia - US              849
California - US           790
North Carolina - US       475
Florida - US              441
Michigan - US             424
Pennsylvania - US         371
Virginia - US             366
Maryland - US             359
New Jersey - US           354
Ohio - US                 331
Illinois - US             297
Louisiana - US            205
Massachusetts - US        187
Connecticut - US          173
Tennessee - US            161
Ontario - CA              138
South Carolina - US       133
Oregon - US               119
Mississippi - US          116
Minnesota - US            109
Washington - US           109
Alabama - US               96
Indiana - US               95
West Virginia - US         56
Missouri - US              54
Washington, D.C. - US      53
Nova Scotia - CA           45
Iowa - US                  35
Delaware - US              30
Quebec - CA                26
Wyoming - US               25
Arizona - 

In [826]:
st = pd.read_csv('state_coordinates.csv')

In [827]:
def find_emoji(tweet):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    emo = re.findall(emoji_pattern, tweet)
    return ''.join(emo)

In [828]:
df.tweet = df.tweet.apply(lambda x: re.sub(find_emoji(x),'',x))

In [829]:
from sklearn.naive_bayes import MultinomialNB

In [830]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [831]:
X = df.tweet
y = df.decoded

In [832]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=1)

In [833]:
ind_x  = list(y_test.index)

In [834]:
le = LabelEncoder()

In [835]:
y_train = le.fit_transform(y_train)
y_test  = le.transform(y_test)

In [836]:
vectorizer = TfidfVectorizer(min_df=10,max_df=0.85)

In [837]:
X_train = vectorizer.fit_transform(X_train)

In [838]:
X_test = vectorizer.transform(X_test)

In [839]:
mNB = MultinomialNB()

In [840]:
mNB.fit(X_train,y_train)

MultinomialNB()

In [841]:
y_pred_mNB = mNB.predict(X_test)

In [842]:
print(classification_report(y_test,y_pred_mNB,zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00         6
           2       0.20      0.10      0.13       315
           3       0.00      0.00      0.00        68
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00       167
           6       0.14      0.06      0.09       341
           7       0.00      0.00      0.00       121
           8       0.00      0.00      0.00        42
           9       0.00      0.00      0.00        17
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00        83
          13       0.65      0.08      0.14       144
          14       0.00      0.00      0.00        76
          15       0.00      0.00      0.00       191
          16       0.00      0.00      0.00        40
          17       0.00    

In [843]:
mNB_result = pd.DataFrame({'y_pred':y_pred_mNB,
              'y_test':y_test,
              'coord':list(df[df.index.isin(ind_x)]['location'])})

In [844]:
from sklearn.ensemble import RandomForestClassifier

In [845]:
RFC = RandomForestClassifier()

In [846]:
RFC.fit(X_train,y_train)

RandomForestClassifier()

In [847]:
y_pred_rfc = RFC.predict(X_test)

In [848]:
print(classification_report(y_test,y_pred_rfc))

              precision    recall  f1-score   support

           0       0.50      0.03      0.05        36
           1       0.00      0.00      0.00         6
           2       0.13      0.19      0.15       315
           3       0.00      0.00      0.00        68
           4       0.00      0.00      0.00         7
           5       0.08      0.06      0.07       167
           6       0.14      0.14      0.14       341
           7       0.22      0.05      0.08       121
           8       0.00      0.00      0.00        42
           9       0.00      0.00      0.00        17
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         4
          12       0.36      0.05      0.09        83
          13       0.25      0.12      0.16       144
          14       0.12      0.03      0.04        76
          15       0.11      0.03      0.05       191
          16       0.11      0.03      0.04        40
          17       0.00    

C:\Users\kenma\Anaconda3\envs\kaggle_cl_challenge\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [849]:
rfc_result = pd.DataFrame({'y_pred':y_pred_rfc,
              'y_test':y_test,
              'coord':list(df[df.index.isin(ind_x)]['location'])})

In [850]:
mNB_result

,y_pred,y_test,coord
0,21,21,"47.528139,-122.197916"
1,21,14,"47.528139,-122.197916"
2,21,13,"47.528139,-122.197916"
3,23,23,"47.528139,-122.197916"
4,21,21,"47.528139,-122.197916"
...,...,...,...
3915,6,6,"36.011464,-83.973806"
3916,21,26,"36.011464,-83.973806"
3917,21,21,"36.011464,-83.973806"
3918,21,21,"42.133812,-87.887853"


In [851]:
from sklearn.linear_model import LogisticRegression

In [852]:
LR = LogisticRegression(solver='lbfgs',max_iter=1000)

In [853]:
LR.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [854]:
y_pred_lr = LR.predict(X_test)

In [855]:
print(classification_report(y_test,y_pred_lr))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00         6
           2       0.15      0.19      0.17       315
           3       0.00      0.00      0.00        68
           4       0.00      0.00      0.00         7
           5       0.09      0.04      0.05       167
           6       0.14      0.16      0.15       341
           7       0.17      0.01      0.02       121
           8       0.00      0.00      0.00        42
           9       0.00      0.00      0.00        17
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         4
          12       1.00      0.02      0.05        83
          13       0.34      0.11      0.17       144
          14       0.00      0.00      0.00        76
          15       0.10      0.01      0.02       191
          16       0.00      0.00      0.00        40
          17       0.00    

C:\Users\kenma\Anaconda3\envs\kaggle_cl_challenge\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [856]:
lr_result = pd.DataFrame({'y_pred':y_pred_lr,
              'y_test':y_test,
              'coord':list(df[df.index.isin(ind_x)]['location'])})

In [857]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()

In [858]:
lsvc.fit(X_train,y_train)

LinearSVC()

In [859]:
y_pred_lsvc = lsvc.predict(X_test)

In [860]:
print(classification_report(y_test,y_pred_lsvc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00         6
           2       0.16      0.17      0.16       315
           3       0.07      0.06      0.06        68
           4       0.00      0.00      0.00         7
           5       0.08      0.08      0.08       167
           6       0.14      0.14      0.14       341
           7       0.12      0.07      0.09       121
           8       0.10      0.05      0.06        42
           9       0.00      0.00      0.00        17
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         4
          12       0.15      0.05      0.07        83
          13       0.16      0.11      0.13       144
          14       0.10      0.05      0.07        76
          15       0.06      0.04      0.05       191
          16       0.07      0.05      0.06        40
          17       0.07    

In [861]:
lsvc_result = pd.DataFrame({'y_pred':y_pred_lsvc,
              'y_test':y_test,
              'coord':list(df[df.index.isin(ind_x)]['location'])})

In [862]:
from sklearn.svm import SVC

In [863]:
svc = SVC()

In [864]:
svc.fit(X_train,y_train)

SVC()

In [865]:
y_pred_svc = svc.predict(X_test)

print(classification_report(y_test,y_pred_svc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00         6
           2       0.15      0.11      0.13       315
           3       0.00      0.00      0.00        68
           4       0.00      0.00      0.00         7
           5       0.33      0.03      0.05       167
           6       0.16      0.08      0.11       341
           7       0.00      0.00      0.00       121
           8       0.00      0.00      0.00        42
           9       0.00      0.00      0.00        17
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         4
          12       1.00      0.01      0.02        83
          13       0.54      0.10      0.16       144
          14       0.00      0.00      0.00        76
          15       0.00      0.00      0.00       191
          16       0.00      0.00      0.00        40
          17       0.00    

C:\Users\kenma\Anaconda3\envs\kaggle_cl_challenge\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [866]:
svc_result = pd.DataFrame({'y_pred':y_pred_svc,
              'y_test':y_test,
              'coord':list(df[df.index.isin(ind_x)]['location'])})

In [867]:
svc_result

,y_pred,y_test,coord
0,21,21,"47.528139,-122.197916"
1,21,14,"47.528139,-122.197916"
2,21,13,"47.528139,-122.197916"
3,23,23,"47.528139,-122.197916"
4,13,21,"47.528139,-122.197916"
...,...,...,...
3915,6,6,"36.011464,-83.973806"
3916,21,26,"36.011464,-83.973806"
3917,21,21,"36.011464,-83.973806"
3918,21,21,"42.133812,-87.887853"


In [868]:
df[df.index==3928]

,user_id,date_time,location,lat,long,tweet,decoded


### Even though accuracy is 23% the model can locate in a twitter user in  America within 

In [869]:
def acc161(data):
    y_161 = []
    for i in data.index:
        if data.y_pred[i] != data.y_test[i]:
            
            y_pred_coord = (st.iloc[data.y_pred[int(i)]][0],
                            st.iloc[data.y_pred[int(i)]][1])
            
            y_test_coord = (float(data.coord[int(i)].split(',')[0]),
                            float(data.coord[int(i)].split(',')[1]))
            
            dist = great_circle(y_pred_coord, y_test_coord).km
            
            if dist <= 161*10:
                y_161.append(data.y_pred[int(i)])
            else:
                y_161.append(data.y_test[int(i)])
        else:
            y_161.append(data.y_pred[int(i)])
    copy = data.copy()
    copy['y_161'] = pd.DataFrame(y_161,columns=['y_161'])
    
    return (print(classification_report(copy.y_161,copy.y_pred)),copy)

In [870]:
p,c = acc161(lsvc_result)

              precision    recall  f1-score   support

           0       0.18      0.27      0.22        15
           1       0.00      0.00      0.00         1
           2       0.75      0.73      0.74       361
           3       0.38      0.53      0.44        40
           4       0.88      0.88      0.88         8
           5       0.88      0.71      0.79       207
           6       0.80      0.72      0.76       400
           7       0.80      0.58      0.67       103
           8       0.71      0.42      0.53        36
           9       0.67      0.20      0.31        10
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.88      0.41      0.56        56
          13       0.90      0.67      0.77       134
          14       0.80      0.60      0.69        55
          15       0.78      0.61      0.69       161
          16       0.85      0.62      0.72        37
          17       0.30    

C:\Users\kenma\Anaconda3\envs\kaggle_cl_challenge\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [871]:
c

,y_pred,y_test,coord,y_161
0,20,21,"47.528139,-122.197916",21
1,27,14,"47.528139,-122.197916",27
2,5,13,"47.528139,-122.197916",13
3,23,23,"47.528139,-122.197916",23
4,21,21,"47.528139,-122.197916",21
...,...,...,...,...
3915,6,6,"36.011464,-83.973806",6
3916,32,26,"36.011464,-83.973806",26
3917,21,21,"36.011464,-83.973806",21
3918,27,21,"42.133812,-87.887853",21


In [872]:
train = pd.read_csv('train.csv')

In [873]:
test = pd.read_csv('test.csv')

In [874]:
def find_emoji(tweet):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    emo = re.findall(emoji_pattern, tweet)
    return ''.join(emo)

train.message =train.message.apply(lambda x: re.sub(find_emoji(x),'',x))
test.message =test.message.apply(lambda x: re.sub(find_emoji(x),'',x))

In [875]:
train_tweets = vectorizer.transform(train.message)

In [876]:
test_tweets = vectorizer.transform(test.message)

In [877]:
train_pred = LR.predict(train_tweets)

In [878]:
test_pred = LR.predict(test_tweets)

In [879]:
Test = pd.DataFrame({'loc_id': test_pred,
                     'location':le.inverse_transform(test_pred)})

In [880]:
Test

,loc_id,location
0,21,New York - US
1,6,Georgia - US
2,21,New York - US
3,31,Texas - US
4,21,New York - US
...,...,...
10541,31,Texas - US
10542,21,New York - US
10543,31,Texas - US
10544,5,Florida - US


In [881]:
Train = pd.DataFrame({'loc_id': train_pred,
                     'location':le.inverse_transform(train_pred)})

In [882]:
Train.location.value_counts()

New York - US          6415
Texas - US             4539
California - US        1948
Georgia - US           1789
Florida - US            332
Maryland - US           216
Michigan - US           171
North Carolina - US     104
Ohio - US                98
New Jersey - US          66
Virginia - US            64
Illinois - US            42
Washington - US          13
Louisiana - US           11
Pennsylvania - US         8
Connecticut - US          2
Massachusetts - US        1
Name: location, dtype: int64

In [883]:
train['location'] = Train.location

In [884]:
train.to_csv('train_with_loc.csv')

In [885]:
test['location'] = Test.location

In [886]:
test.to_csv('test_with_loc.csv')

In [887]:
model_save_path = "location_extractor.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(svc,file)

In [ ]:
#https://docs.streamlit.io/en/stable/tutorial/create_a_data_explorer_app.html

In [ ]:
#https://awesome-streamlit.org/ 

In [ ]:
#https://github.com/Jcharis/Streamlit_DataScience_Apps/tree/master/NLP_App_with_Streamlit_Python

In [ ]:
experiment.end()

In [ ]:
experiment.display()